# Overview

This is a simple demo of how to use Mesh plugin for TensorBoard. The demo will load static triangulated mesh (in PLY format), create a mesh summary with it and then display in TensorBoard.

# Setup Imports

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Uninstall tensorboard and tensorflow
!pip uninstall -q -y tensorboard
!pip uninstall -q -y tensorflow
# Install nightly TensorFlow with nightly TensorBoard.
!pip install tf-nightly
# Install trimesh lib to read .PLY files.
!pip freeze | grep -qF 'trimesh==' || pip install trimesh

%load_ext tensorboard

import os

import numpy as np
import tensorflow as tf
import trimesh
import tensorboard

from tensorboard.plugins.mesh import summary as mesh_summary

sample_mesh = 'https://storage.googleapis.com/tensorflow-graphics/tensorboard/test_data/ShortDance07_a175_00001.ply'
log_dir = '/tmp/mesh_demo'
batch_size = 1

!rm -rf /tmp/mesh_demo

# Read sample .PLY files

In [0]:

# Camera and scene configuration.
config_dict = {
    'camera': {'cls': 'PerspectiveCamera', 'fov': 75},
    'lights': [
        {
          'cls': 'AmbientLight', 
          'color': '#ffffff',
          'intensity': 0.75,
        }, {
          'cls': 'DirectionalLight',
          'color': '#ffffff',
          'intensity': 0.75,
          'position': [0, -1, 2],
        }],
    'material': {
      'cls': 'MeshStandardMaterial',
      'roughness': 1,
      'metalness': 0
    }
}

# Read all sample PLY files.
mesh = trimesh.load_remote(sample_mesh)    
vertices = np.array(mesh.vertices)
# Currently only supports RGB colors.
colors = np.array(mesh.visual.vertex_colors[:, :3])
faces = np.array(mesh.faces)

# Add batch dimension, so our data will be of shape BxNxC.
vertices = np.expand_dims(vertices, 0)
colors = np.expand_dims(colors, 0)
faces = np.expand_dims(faces, 0)

# Create summaries and session

In [3]:
# Create data placeholders of the same shape as data itself.
vertices_tensor = tf.placeholder(tf.float32, vertices.shape)
faces_tensor = tf.placeholder(tf.int32, faces.shape)
colors_tensor = tf.placeholder(tf.int32, colors.shape)

meshes_summary = mesh_summary.op(
    'mesh_color_tensor', vertices=vertices_tensor, faces=faces_tensor,
    colors=colors_tensor, config_dict=config_dict)

# Create summary writer and session.
writer = tf.summary.FileWriter(log_dir)
sess = tf.Session()

W0509 14:00:39.764862 140689019053952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tensorboard/plugins/mesh/summary.py:59: The name tf.summary.tensor_summary is deprecated. Please use tf.compat.v1.summary.tensor_summary instead.

W0509 14:00:39.776251 140689019053952 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tensorboard/plugins/mesh/summary.py:125: The name tf.summary.merge is deprecated. Please use tf.compat.v1.summary.merge instead.



# Run the model, save summaries to disk

In [0]:
summaries = sess.run([meshes_summary], feed_dict={
    vertices_tensor: vertices,
    faces_tensor: faces,
    colors_tensor: colors,
})
# Save summaries.
for summary in summaries:
  writer.add_summary(summary)

# TensorBoard

In [0]:
%tensorboard --logdir=/tmp/mesh_demo